In [1]:
import os
from docx import Document
from docx.opc.constants import RELATIONSHIP_TYPE as RT
import boto3
import json
import csv
from dotenv import load_dotenv
import json
import os
import pandas as pd
import re
import requests
load_dotenv()


s3 = boto3.client('s3')


response = requests.get('https://randomuser.me/api/')
data = response.json()

# Access the data returned from the API
results = data['results']
for result in results:
    first_name = result['name']['first']
    last_name = result['name']['last']
    age = result['age']
    gender = result['gender']
    lattitude = result['location']['coordinates']['latitude']
    longitude = result['location']['coordinates']['longitude']

    firehose_client = boto3.client('firehose')

    # Convert the data dictionary to JSON string
    json_data = json.dumps(data)

    # Specify the delivery stream name
    delivery_stream_name = 'your_delivery_stream_name'

    # Put the JSON data to the delivery stream
    response = firehose_client.put_record(
        DeliveryStreamName=delivery_stream_name,
        Record={
            'Data': json_data.encode('utf-8')
        }
    )

    # Check if the record was successfully put
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        print('JSON data successfully loaded to Kinesis Firehose')
    else:
        print('Failed to load JSON data to Kinesis Firehose')

    
    # print(f"Name: {name}, Email: {email}")




Name: Alessandro, Email: alessandro.gauthier@example.com


In [ ]:


def upload_to_s3(file_name, bucket, object_name=None):
    mime_type = mime.from_file(file_name) 
    s3.upload_file(
        file_name, 
        bucket, 
        object_name or file_name,
        ExtraArgs={
        'ContentType': mime_type
        })
    return f"https://{bucket}.s3.amazonaws.com/{object_name or file_name}"


def list_image_files(directory):
    image_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
                image_files.append(os.path.join(root, file))
    return image_files


def get_image_info_from_csv(df_csv, orig_filename):
    
    matching_line = df_csv[df_csv['file'] == orig_filename]
    image_description = ""
    if not matching_line.empty:
        image_description = matching_line.iloc[0]['description'] or ''
    return image_description


if os.path.exists( INPUT_PATH ):

    image_data = []
    image_files = list_image_files(INPUT_PATH)
    df_csv = pd.read_csv(CSV_INFO)    
    for file in image_files:
        try: 
            filename = os.path.basename(file)
            folder = os.path.dirname(file)
            desc = get_image_info_from_csv(df_csv, filename)
            desc_path = os.path.join(folder, os.path.splitext(filename)[0])
            path_desc = ' '.join(re.split(r'[^a-zA-Z@]+', desc_path))
            desc = f"{desc} {path_desc}".strip()
            url = upload_to_s3(file, IMAGE_BUCKET, filename)
            csv_file = f'{filename}.csv'

            
            with open(csv_file, 'w+', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=[ 'image_description', 'image_url',])
                writer.writeheader()
                writer.writerow({'image_description': f'{desc}', 'image_url': url})
            upload_to_s3(csv_file, IMAGE_REF_BUCKET)
            os.remove(csv_file)
        except Exception as e:
            print(f"Error processing file {file}: {e}")
        
        print(f"======================================================")
        print(f"Processed... '{file.name}'")
        print(f"\timage uploaded s3://{IMAGE_BUCKET}/{filename}")
        print(f"\tdescription created '{desc}'")
        print(f"\tcsv uploaded s3://{IMAGE_REF_BUCKET}/{csv_file}")
        
else:
    print(f"Directory '{INPUT_PATH}' does not exist.")

